Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock,
Germany

# Data Driven Audio Signal Processing - A Tutorial with Computational Examples

Master Course #24512

- lecture: https://github.com/spatialaudio/data-driven-audio-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise

Feel free to contact lecturer frank.schultz@uni-rostock.de

# Principal Component Analysis (PCA)

Example in 3D, data matrix with M samples in 3 features

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import svd, diagsvd
from statsmodels.multivariate.pca import PCA

In [ ]:
matplotlib_widget_flag = True

In [ ]:
if matplotlib_widget_flag:
    %matplotlib widget

## Create Data and Plot in Original Coordinate System

In [ ]:
rng = np.random.default_rng(1)  # be careful when changing seed
# SVD U and V vectors might then need reflections
# to match with statsmodels results
# also the colored data points to indicate the PC directions might not work

# we construct 3 features somehow linearly related drawn from normal PDF
# the PCA usually assumes mean-free columns, we can design them this way:
mean = [0, 0, 0]
cov = [[3, 2, 0], [2, 3, 1], [0, 1, 1]]

M = 200  # no of samples
x, y, z = rng.multivariate_normal(mean, cov, M).T
X = np.array([x, y, z]).T
# center data to origin, remove mean
X = X - np.mean(X, axis=0)
N = X.shape[1]

print("rank", np.linalg.matrix_rank(X))
print("X.shape", X.shape)

# index for specific data points to plot with specific colors
# this helps to identify potential reflections of U and V space vectors
di1 = 36
di2 = 23
di3 = 57

fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.plot(X[:, 0], X[:, 1], X[:, 2], "x", color="gray")
ax.plot(X[di1, 0], X[di1, 1], X[di1, 1], "C3x", ms=10, mew=3)
ax.plot(X[di2, 0], X[di2, 1], X[di2, 2], "C2x", ms=10, mew=3)
ax.plot(X[di3, 0], X[di3, 1], X[di3, 2], "C0x", ms=10, mew=3)

ax.axis("square")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.set_zlim(-6, 6)
ax.set_xlabel("feature 1")
ax.set_ylabel("feature 2")
ax.set_zlabel("feature 3")
ax.set_title("original data in original coordinate system")
ax.grid(True)

## Calculate Principal Component Analysis (PCA)

In [ ]:
# we work on matrix X directly (so standardize=False), and we already
# made it mean free (so demean=False), normalize=False to give us
# data that is nicely connected to SVD data
# use pca from statsmodels.multivariate.pca
pca = PCA(X, ncomp=3, standardize=False, demean=False, normalize=False)
# using SVD
[U, s, Vh] = svd(X, full_matrices=False)
S = diagsvd(s, N, N)
V = Vh.conj().T

# we use the abbreviation PC for 'principal component'

pcs = U @ S  # known as PC, PC signals, PC factors, PC scores
pcl = V  # known PC loadings, PC coefficients
var_pcs = np.var(pcs, axis=0)  # explained variances
# note that sometimes V.T is called loadings, coefficients
# check if statsmodels pca and our manual SVD-based PCA produce same results:
print(np.allclose(pca.scores, pcs))
print(np.allclose(pca.coeff.T, V))

## Indicate the Directions of the PCs (Right Sing Vectors in V, PC Loadings)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.plot(X[:, 0], X[:, 1], X[:, 2], "x", color="gray")
ax.plot(X[di1, 0], X[di1, 1], X[di1, 2], "C3x", mew=3)
ax.plot(X[di2, 0], X[di2, 1], X[di2, 2], "C2x", mew=3)
ax.plot(X[di3, 0], X[di3, 1], X[di3, 2], "C0x", mew=3)

# note that not necessarily a right-hand system is spanned
# draw direction of first PC axis -> this is the first col vec of V
ax.plot(
    [0, V[0, 0]],
    [0, V[1, 0]],
    [0, V[2, 0]],
    "C3",
    lw=3,
    label="direction of 1st PC == 1st col in V, 1st right sing vec",
)
# draw direction of first PC axis with magnitude of variance of first PC score
ax.plot(
    [0, var_pcs[0] * V[0, 0]],
    [0, var_pcs[0] * V[1, 0]],
    [0, var_pcs[0] * V[2, 0]],
    "C3", lw=0.75)
# draw direction of second PC axis -> this is the second col vec of V
ax.plot(
    [0, V[0, 1]],
    [0, V[1, 1]],
    [0, V[2, 1]],
    "C2",
    lw=3,
    label="direction of 2nd PC == 2nd col in V, 2nd right sing vec",
)
ax.plot(
    [0, var_pcs[1] * V[0, 1]], 
    [0, var_pcs[1] * V[1, 1]], 
    [0, var_pcs[1] * V[2, 1]], 
    "C2", lw=0.75)
# draw direction of third PC axis -> this is the third col vec of V
ax.plot(
    [0, V[0, 2]],
    [0, V[1, 2]],
    [0, V[2, 2]],
    "C0",
    lw=3,
    label="direction of 3rd PC == 3rd col in V, 3rd right sing vec",
)
ax.plot(
    [0, var_pcs[2] * V[0, 2]], 
    [0, var_pcs[2] * V[1, 2]], 
    [0, var_pcs[2] * V[2, 2]], 
    "C0", lw=0.75)

ax.axis("square")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.set_zlim(-6, 6)
ax.set_xlabel("original feature 1")
ax.set_ylabel("original feature 2")
ax.set_zlabel("original feature 3")
ax.set_title("original data in original coordinate system")
ax.legend()
ax.grid(True)

## Plot Data in PC Coordinate System

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.plot(pcs[:, 0], pcs[:, 1], pcs[:, 2], "x", color="gray")
ax.plot(pcs[di1, 0], pcs[di1, 1], pcs[di1, 2], "C3x", mew=3)
ax.plot(pcs[di2, 0], pcs[di2, 1], pcs[di2, 2], "C2x", mew=3)
ax.plot(pcs[di3, 0], pcs[di3, 1], pcs[di3, 2], "C0x", mew=3)

# note that not necessarily a right-hand system is spanned
# but we must span the same axis order as above
# draw direction of first PC axis
ax.plot([0, 1], [0, 0], [0, 0], "C3", lw=3, label="direction of 1st PC")
ax.plot([0, var_pcs[0]], [0, 0], [0, 0], "C3", lw=0.75)
# draw direction of second PC axis
ax.plot([0, 0], [0, 1], [0, 0], "C2", lw=3, label="direction of 2nd PC")
ax.plot([0, 0], [0, var_pcs[1]], [0, 0], "C2", lw=0.75)
# draw direction of third PC axis
ax.plot([0, 0], [0, 0], [0, 1], "C0", lw=3, label="direction of 3rd PC")
ax.plot([0, 0], [0, 0], [0, var_pcs[2]], "C0", lw=0.75)

ax.axis("square")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.set_zlim(-6, 6)
ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 3")
ax.set_title("data in PC coordinate system")
ax.legend()
ax.grid(True)

## Variances

For rank r data matrix, the data is rotated (and potentially reflected in some axes) such that along the axis of PC 1 most variance occurs (most data spread), whereas along the axis of the r-th PC (last PC, here in the case it is PC2) has fewest variance.
Generally, var(PC1)>var(PC2)>...>var(PCr) just as the sorting of the singular values in the SVD. Recall how we calculated `pcs`...

In [ ]:
print(np.var(pcs[:, 0], ddof=1))
print(np.var(pcs[:, 1], ddof=1))
print(np.var(pcs[:, 2], ddof=1))
print(var_pcs)

We compare these variances to the the variances of the original features

In [ ]:
print(np.var(X[:, 0], ddof=1))
print(np.var(X[:, 1], ddof=1))
print(np.var(X[:, 2], ddof=1))

Note, that we don't lose variance, we just distribute them in another way.

In [ ]:
print(
    np.var(pcs[:, 0], ddof=1)
    + np.var(pcs[:, 1], ddof=1)
    + np.var(pcs[:, 2], ddof=1)
)
print(
    np.var(X[:, 0], ddof=1)
    + np.var(X[:, 1], ddof=1)
    + np.var(X[:, 2], ddof=1)
)

That's the fundamental idea of shaping the data to re-sort variances, here using the special case of PCA, i.e. projection of data onto vector of an orthonormal basis...our well known column space spanned in U matrix of the SVD. Variance is explained by their corresponding singular values. Recall how we calculated `pcs`...

1st PC signal explains 74.63 % of total variance:

In [ ]:
np.var(pcs[:, 0], ddof=1) / (
    np.var(pcs[:, 0], ddof=1)
    + np.var(pcs[:, 1], ddof=1)
    + np.var(pcs[:, 2], ddof=1)
) * 100

2nd PC signal explains next 21.93 % of total variance: 

In [ ]:
np.var(pcs[:, 1], ddof=1) / (
    np.var(pcs[:, 0], ddof=1)
    + np.var(pcs[:, 1], ddof=1)
    + np.var(pcs[:, 2], ddof=1)
) * 100

3nd PC signal explains the remaining 3.44 % of total variance: 

In [ ]:
np.var(pcs[:, 2], ddof=1) / (
    np.var(pcs[:, 0], ddof=1)
    + np.var(pcs[:, 1], ddof=1)
    + np.var(pcs[:, 2], ddof=1)
) * 100

## Truncated SVD

matrix rank reduction

- for `r_des = 2` we reduce data information to a **plane** in **3D space**
- for `r_des = 1` we reduce data information to a **line** in **3D space**


In [ ]:
r_des = 2  # 1 or 2 

# SVD mindset:
# we could make a sum of outer products, i.e. sum of rank-1 matrices
X_rank_red = np.zeros((M, N))
for i in range(r_des):
    # either
    # X_rank_red += S[i,i] * U[:,i][:,None] @ V[:,i][:,None].T
    # or
    X_rank_red += s[i] * np.outer(U[:, i], V[:, i])

# PCA mindset:
# we might also use the PC signals and set intended PC loadings to zero
X_rank_red2 = np.zeros((M, N))
pcl_rank_red = np.copy(pcl)
pcl_rank_red[:, r_des:] = 0
X_rank_red2 = pcs @ pcl_rank_red.conj().T

np.allclose(X_rank_red, X_rank_red2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.plot(
    X_rank_red2[:, 0], X_rank_red2[:, 1], X_rank_red2[:, 2], "x", color="gray"
)
ax.plot(
    X_rank_red2[di1, 0], X_rank_red2[di1, 1], X_rank_red2[di1, 2], "C3x", mew=3
)
ax.plot(
    X_rank_red2[di2, 0], X_rank_red2[di2, 1], X_rank_red2[di2, 2], "C2x", mew=3
)
ax.plot(
    X_rank_red2[di3, 0], X_rank_red2[di3, 1], X_rank_red2[di3, 2], "C0x", mew=3
)

ax.axis("square")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.set_zlim(-6, 6)
ax.set_xlabel("new feature 1 for rank reduction")
ax.set_ylabel("new feature 2 for rank reduction")
ax.set_zlabel("new feature 2 for rank reduction")
ax.set_title("rank {0:d} approximation of data".format(r_des))
ax.grid(True)

## Dimensionality Reduction after PCA

- for `dim_des = 2` we reduce data to a **plane** in **2D space**, it is though plotted in 3D (third variable is zero) for convenience
- for `dim_des = 1` we reduce data to a **line** in **1D space**, it is though plotted in 3D (second and third variable is zero) for convenience

In [ ]:
dim_des = 2  # 1 or 2

# PCA mindset
X_dim_red = np.zeros((M, dim_des))
X_dim_red = pcs[:, :dim_des]
print('original matrix shape     ', X.shape)
print('matrix shape after dim red', X_dim_red.shape)

X_dim_red_plot = np.zeros((M, N))
X_dim_red_plot[:, :dim_des] = pcs[:, :dim_des]

# check with SVD mindset
np.allclose((U @ S)[:,:dim_des], X_dim_red)

In [ ]:
# for convenience we plot data here in 3D plot
# note however that X_dim_red_plot[:,2] is precisely zero if
# dim_des = 2 was chosen
# but the dimensionality reduction actually yields a matrix with smaller
# dimension, cf. shape of X_dim_red !!!
fig = plt.figure()
ax = fig.add_subplot(111, projection="3d")

ax.plot(
    X_dim_red_plot[:, 0],
    X_dim_red_plot[:, 1],
    X_dim_red_plot[:, 2],
    "x",
    color="gray",
)
ax.plot(
    X_dim_red_plot[di1, 0],
    X_dim_red_plot[di1, 1],
    X_dim_red_plot[di1, 2],
    "C3x",
    mew=3,
)
ax.plot(
    X_dim_red_plot[di2, 0],
    X_dim_red_plot[di2, 1],
    X_dim_red_plot[di2, 2],
    "C2x",
    mew=3,
)
ax.plot(
    X_dim_red_plot[di3, 0],
    X_dim_red_plot[di3, 1],
    X_dim_red_plot[di3, 2],
    "C0x",
    mew=3,
)

# note that we don't necessarily need to span a right-hand system
# but we should span the same axis order as above
# draw direction of first PC axis
# ax.plot([0, 1], [0, 0], [0, 0], 'C3', lw=3)
ax.plot([0, 6], [0, 0], [0, 0], "C3", lw=0.5, label="direction of 1st PC")
# draw direction of second PC axis
# ax.plot([0, 0], [0, 1], [0, 0], 'C2', lw=3)
ax.plot([0, 0], [0, 6], [0, 0], "C2", lw=0.5, label="direction of 2nd PC")
# draw direction of third PC axis
# ax.plot([0, 0], [0, 0], [0, 1], 'C0', lw=3)
ax.plot([0, 0], [0, 0], [0, 6], "C0", lw=0.5, label="direction of 3rd PC")


ax.axis("square")
ax.set_xlim(-6, 6)
ax.set_ylim(-6, 6)
ax.set_zlim(-6, 6)
ax.set_xlabel("PC 1")
ax.set_ylabel("PC 2")
ax.set_zlabel("PC 3")
plt.title("PCA data dimensionality reduction to {0:d}D".format(r_des))
ax.legend()
ax.grid(True)

print("reduced to dimension", X_dim_red.shape)

## Copyright

- the notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources)
- feel free to use the notebooks for your own purposes
- the text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/)
- the code of the IPython examples is licensed under the [MIT license](https://opensource.org/licenses/MIT)
- please attribute the work as follows: *Frank Schultz, Data Driven Audio Signal Processing - A Tutorial Featuring Computational Examples, University of Rostock* ideally with relevant file(s), github URL https://github.com/spatialaudio/data-driven-audio-signal-processing-exercise, commit number and/or version tag, year.